<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>
<h1 align=center><font size = 8>Data Science Capstone Project</font></h1>

In [1]:
try:
    import pandas as pd
    import numpy as np
except:
    !conda install -c anaconda pandas numpy --yes 
    import pandas as pd
    import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
try:
    from geopy.geocoders import Nominatim # to convert address into latitude and longitude
except:
    !conda install -c conda-forge geopy --yes 
    from geopy.geocoders import Nominatim
try:
    import folium # plotting library
except:
    !conda install -c conda-forge folium --yes 
    import folium
from bs4 import BeautifulSoup
import urllib.request
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import geojson

## Read Zillow geodata

In [2]:
with open('e:/downloads/zillow-neighborhoods-FortWorth.geojson') as json_data:
    fortworth_neighborhood_data = geojson.load(json_data)
with open('e:/downloads/zillow-neighborhoods-Dallas.geojson') as json_data:
    dallas_neighborhood_data = geojson.load(json_data)
print(type(fortworth_neighborhood_data), type(dallas_neighborhood_data))

<class 'geojson.feature.FeatureCollection'> <class 'geojson.feature.FeatureCollection'>


In [4]:
fortworth_neighborhood_data = fortworth_neighborhood_data['features']
df = pd.DataFrame(columns=['city','name','regionid','geo_point_2d','county','state'])
for entry in fortworth_neighborhood_data:
    df = df.append( entry['properties'],ignore_index=True)
dallas_neighborhood_data = dallas_neighborhood_data['features']
for entry in dallas_neighborhood_data:
    df = df.append( entry['properties'],ignore_index=True)
df = df.rename(columns={'name': 'Neighborhood'})
df.head()

,city,Neighborhood,regionid,geo_point_2d,county,state
0,Fort Worth,Briercliff,422763,"[32.64714146486756, -97.43035652827258]",Tarrant,TX
1,Fort Worth,Fairmount,233172,"[32.724372978687654, -97.33766189387364]",Tarrant,TX
2,Fort Worth,Willow Creek,207722,"[32.63110091743264, -97.3434603767844]",Tarrant,TX
3,Fort Worth,Shaw Clarke,422871,"[32.70111450603923, -97.33719612060821]",Tarrant,TX
4,Fort Worth,Carver Heights,422820,"[32.72505076405124, -97.22932078449055]",Tarrant,TX


Split up geo_point_2d into Latitude and Longitude

In [5]:
print(df.shape)
df = df.dropna(subset=['geo_point_2d'])
print(df.shape)
df2 = pd.DataFrame(df["geo_point_2d"].tolist(), columns=['Latitude','Longitude']) #pd.DataFrame(df['geo_point_2d'].tolist(),index=df.index)
df = pd.concat([df, df2], axis=1)
df.drop(['geo_point_2d','regionid'],axis=1,inplace=True)
df.head()

(425, 6)
(425, 6)


,city,Neighborhood,county,state,Latitude,Longitude
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357
1,Fort Worth,Fairmount,Tarrant,TX,32.724373,-97.337662
2,Fort Worth,Willow Creek,Tarrant,TX,32.631101,-97.343460
3,Fort Worth,Shaw Clarke,Tarrant,TX,32.701115,-97.337196
4,Fort Worth,Carver Heights,Tarrant,TX,32.725051,-97.229321


Extract Dallas-Fort Worth

In [8]:
#dfw = df[df['city'].isin(['Dallas', 'Fort Worth'])]
dfw = df
print(dfw.shape)
dfw

(425, 6)


,city,Neighborhood,county,state,Latitude,Longitude
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357
1,Fort Worth,Fairmount,Tarrant,TX,32.724373,-97.337662
2,Fort Worth,Willow Creek,Tarrant,TX,32.631101,-97.343460
3,Fort Worth,Shaw Clarke,Tarrant,TX,32.701115,-97.337196
4,Fort Worth,Carver Heights,Tarrant,TX,32.725051,-97.229321
5,Fort Worth,Santa Fe Enclave,Tarrant,TX,32.868511,-97.319341
6,Fort Worth,Park Place,Tarrant,TX,32.879230,-97.281318
7,Fort Worth,Harbour Points,Tarrant,TX,32.892593,-97.452058
8,Fort Worth,Alexandra Meadows,Tarrant,TX,32.857766,-97.329245
9,Fort Worth,Chapel Creek,Tarrant,TX,32.735927,-97.504439


## Create a map of DFW Neighborhoods

In [7]:
# create map - use Arlington for map lat/long
address = 'Arlington, TX'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Arlington are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Arlington are 32.701938999999996, -97.10562379033699.


In [10]:
# create map of Toronto using latitude and longitude values
map_dfw = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighborhood in zip(dfw['Latitude'], dfw['Longitude'], dfw['city'], df['Neighborhood']):
    label = '{}, {}, {}, {}'.format(neighborhood, city, lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dfw)  
    
map_dfw

# Get Neighborhood Venues

In [11]:
CLIENT_ID = 'YPF4NY2VJMYEMROHWYBYVI11IZKQ0H4EDECKGJR0XDYRWE4M' # your Foursquare ID
CLIENT_SECRET = 'LI202HN0RHIHWIIQQPWSYIPUQE14DLGSNALI21B5XGV4E31Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:    
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('\tProblem getting info')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
dfw_venues = getNearbyVenues(names=dfw['Neighborhood'],
                                   latitudes=dfw['Latitude'],
                                   longitudes=dfw['Longitude']
                                  )
print('There are {} uniques categories.'.format(len(dfw_venues['Venue Category'].unique())))

Briercliff
Fairmount
Willow Creek
Shaw Clarke
Carver Heights
Santa Fe Enclave
Park Place
Harbour Points
Alexandra Meadows
Chapel Creek
Stop Six
Mitchell Boulevard
Jennings - May St. Louis
Hollow Hills
Arlington Heights
Fossil Hill Estates
Overton South
Oakridge Terrace
Scenic Bluff
Sendera Ranch
Diamond Hill - Jarvis
Hamlet
Basswood Park
Sunset Heights South
The Crossing of Fossil Creek
Sylvan Heights West
South Edgewood
Panther Heights
Hulen Heights
Marine Park
Vista Ridge
Basswood Village
Paschal
Lake Country
Worth Heights
Burton Hill Trinity Trails
Stone Meadow
Villages of Woodland Springs
Crestline Area
Candle Ridge West
Caville
Central Chapel Creek
Westcliff
The Trails of Marine Creek
Sunset Hills
Far East
Park Hill
Glencrest
Emory Place
Alamo Heights
Ridglea Hills
South Meadows Crossing
Chadwick Farms
El Poly Pyramid
Eagle Ranch
Poynter Crossing
Westpoint
Deer Creek North
Bonnie Brae
Bently Village
Brittany Place
Bomber Heights
Northbrook
Western Hills North
Scenic Shores
The Vin

In [14]:
dfw_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alamo Heights,5,5,5,5,5,5
Alexandra Meadows,2,2,2,2,2,2
"Almeta,Bonita, Bella Vista",1,1,1,1,1,1
Altemesa East,2,2,2,2,2,2
Apollo Arapaho & Camelot,4,4,4,4,4,4
Arapaho,2,2,2,2,2,2
Arcadia Park,6,6,6,6,6,6
Arlington Heights,5,5,5,5,5,5
Arts District,35,35,35,35,35,35


# Analyze Neighborhoods

In [15]:
# one hot encoding
dfw_onehot = pd.get_dummies(dfw_venues[['Venue Category']], prefix="", prefix_sep="")
# There is Neighborhood in the venue categories, so we drop that column
try:
    del dfw_onehot['Neighborhood']
except:
    print('No venues called Neighborhood')
# add neighborhood column back to dataframe
dfw_onehot['Neighborhood'] = dfw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfw_onehot.columns[-1]] + list(dfw_onehot.columns[:-1])
dfw_onehot = dfw_onehot[fixed_columns]
dfw_onehot.head()

No venues called Neighborhood


,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Stadium,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Insurance Office,Internet Cafe,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighthouse,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Watch Shop,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Fairmount,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [53]:
dfw_grouped = dfw_onehot.groupby('Neighborhood').mean().reset_index()
dfw_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Stadium,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Insurance Office,Internet Cafe,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighthouse,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Watch Shop,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Alamo Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

# Group most common venues in each neighborhood

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dfw_grouped['Neighborhood']

for ind in np.arange(dfw_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfw_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Heights,Thrift / Vintage Store,Taco Place,Diner,Sandwich Place,Thai Restaurant,Zoo Exhibit,Event Service,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
1,Alexandra Meadows,Park,Pool,Zoo Exhibit,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
2,"Almeta,Bonita, Bella Vista",Food,Concert Hall,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
3,Altemesa East,Donut Shop,Discount Store,Dry Cleaner,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
4,Apollo Arapaho & Camelot,Construction & Landscaping,Southern / Soul Food Restaurant,Park,Spa,Farm,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory


In [56]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6

dfw_grouped_clustering = dfw_grouped.drop('Neighborhood', axis=1)
if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted.drop('Cluster Labels',axis=1, inplace=True)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfw_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dfw_merged = dfw

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfw_merged = dfw_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dfw_merged.head() # check the last columns!

,city,Neighborhood,county,state,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fort Worth,Fairmount,Tarrant,TX,32.724373,-97.337662,3.0,Beer Garden,Garden,Bed & Breakfast,Liquor Store,Furniture / Home Store,Zoo Exhibit,Event Service,Event Space,Factory,Farm
2,Fort Worth,Willow Creek,Tarrant,TX,32.631101,-97.343460,3.0,Fried Chicken Joint,Discount Store,Zoo Exhibit,Dry Cleaner,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
3,Fort Worth,Shaw Clarke,Tarrant,TX,32.701115,-97.337196,4.0,Playground,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
4,Fort Worth,Carver Heights,Tarrant,TX,32.725051,-97.229321,0.0,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory


## identify neighborhoods with invalid cluster labels

In [61]:
idx = pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').isna()
print('Number of neighborhoods with invalid Cluster Labels: ',len(dfw_merged[idx]), ' out of ',len(dfw_merged.index))
neighborhoods_venues_sorted[idx]

Number of neighborhoods with invalid Cluster Labels:  54  of  425


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Alamo Heights,Thrift / Vintage Store,Taco Place,Diner,Sandwich Place,Thai Restaurant,Zoo Exhibit,Event Service,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
16,3,Bellaire Heights,BBQ Joint,Zoo Exhibit,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
17,3,Bellaire Park North,Jewelry Store,Trail,Asian Restaurant,Zoo Exhibit,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space
27,4,Brentwood Village,Home Service,Playground,Factory,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
30,3,Brittany Place,Farmers Market,Zoo Exhibit,Concert Hall,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farm,Factory
33,3,Buckingham North,Pizza Place,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
47,3,Carrollton Summertree,Home Service,Dry Cleaner,Park,Playground,Trail,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
59,5,Chapel Creek,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
75,2,Coventry Hills,Pool,Dive Bar,Fast Food Restaurant,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
78,3,Creek Valley,Discount Store,Zoo Exhibit,Dry Cleaner,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm


## Map clusters--removing rows with invalid cluster labels and converting cluster labels to int before we map them

In [45]:

# first, we have to drop non-numeric cluster labels and change them to int
dfw_merged = dfw_merged[pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').notnull()]
dfw_merged['Cluster Labels'] = dfw_merged['Cluster Labels'].astype('int')
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfw_merged['Latitude'], dfw_merged['Longitude'], dfw_merged['Neighborhood'], dfw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 0, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Fort Worth,Carver Heights,0,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
8,Fort Worth,Alexandra Meadows,0,Park,Pool,Zoo Exhibit,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
23,Fort Worth,Sunset Heights South,0,Park,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
54,Fort Worth,Eagle Ranch,0,Italian Restaurant,Park,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
101,Fort Worth,Chisholm Ridge,0,Park,Fast Food Restaurant,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Farmers Market,Farm,Factory
126,Fort Worth,Candleridge,0,Lake,Park,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
146,Fort Worth,Oakhurst,0,Park,Construction & Landscaping,Salvadoran Restaurant,Deli / Bodega,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
148,Fort Worth,South Hills,0,Park,Rental Service,Zoo Exhibit,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Factory
172,Fort Worth,North Greenbriar,0,Park,Disc Golf,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
176,Fort Worth,FW Vista West,0,Park,Electronics Store,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm


In [22]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 1, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Fort Worth,Fairmount,1,Beer Garden,Garden,Bed & Breakfast,Liquor Store,Furniture / Home Store,Zoo Exhibit,Event Service,Event Space,Factory,Farm
5,Fort Worth,Santa Fe Enclave,1,Road,Mexican Restaurant,Bagel Shop,Ice Cream Shop,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space
7,Fort Worth,Harbour Points,1,Harbor / Marina,American Restaurant,Farm,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Zoo Exhibit
9,Fort Worth,Chapel Creek,1,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
10,Fort Worth,Stop Six,1,Furniture / Home Store,Zoo Exhibit,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
11,Fort Worth,Mitchell Boulevard,1,Fast Food Restaurant,Big Box Store,Grocery Store,Department Store,Discount Store,Optical Shop,Clothing Store,Sandwich Place,Chinese Restaurant,Electronics Store
12,Fort Worth,Jennings - May St. Louis,1,Burger Joint,Gas Station,Discount Store,Taco Place,Factory,Ethiopian Restaurant,Event Service,Event Space,Zoo Exhibit,Egyptian Restaurant
13,Fort Worth,Hollow Hills,1,Health Food Store,Other Repair Shop,Zoo Exhibit,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Farm
14,Fort Worth,Arlington Heights,1,Concert Hall,Ethiopian Restaurant,Convenience Store,Park,Community Center,Hawaiian Restaurant,Hardware Store,Field,Fast Food Restaurant,Farmers Market
15,Fort Worth,Fossil Hill Estates,1,Sandwich Place,Auto Workshop,Coffee Shop,Farmers Market,Zoo Exhibit,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm


In [35]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 2, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Fort Worth,Fairmount,2,Beer Garden,Garden,Bed & Breakfast,Liquor Store,Furniture / Home Store,Zoo Exhibit,Event Service,Event Space,Factory,Farm
2,Fort Worth,Willow Creek,2,Fried Chicken Joint,Discount Store,Zoo Exhibit,Dry Cleaner,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
5,Fort Worth,Santa Fe Enclave,2,Road,Mexican Restaurant,Bagel Shop,Ice Cream Shop,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space
7,Fort Worth,Harbour Points,2,Harbor / Marina,American Restaurant,Farm,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Zoo Exhibit
10,Fort Worth,Stop Six,2,Furniture / Home Store,Zoo Exhibit,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
11,Fort Worth,Mitchell Boulevard,2,Fast Food Restaurant,Big Box Store,Grocery Store,Department Store,Discount Store,Optical Shop,Clothing Store,Sandwich Place,Chinese Restaurant,Electronics Store
12,Fort Worth,Jennings - May St. Louis,2,Burger Joint,Gas Station,Discount Store,Taco Place,Factory,Ethiopian Restaurant,Event Service,Event Space,Zoo Exhibit,Egyptian Restaurant
13,Fort Worth,Hollow Hills,2,Health Food Store,Other Repair Shop,Zoo Exhibit,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Farm
14,Fort Worth,Arlington Heights,2,Concert Hall,Ethiopian Restaurant,Convenience Store,Park,Community Center,Hawaiian Restaurant,Hardware Store,Field,Fast Food Restaurant,Farmers Market
15,Fort Worth,Fossil Hill Estates,2,Sandwich Place,Auto Workshop,Coffee Shop,Farmers Market,Zoo Exhibit,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm


In [24]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 3, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Fort Worth,Park Place,3,Home Service,Bus Station,Garden Center,Dry Cleaner,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
31,Fort Worth,Basswood Village,3,Home Service,Construction & Landscaping,Convenience Store,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farmers Market
48,Fort Worth,Emory Place,3,Accessories Store,Home Service,History Museum,Dog Run,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
55,Fort Worth,Poynter Crossing,3,Home Service,Playground,Pool,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farmers Market
97,Fort Worth,Waterview at Enchanted Bay,3,Home Service,Furniture / Home Store,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
114,Fort Worth,Highland Hills,3,Home Service,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory,Event Space
155,Fort Worth,Glenwood Triangle,3,Home Service,Brewery,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
190,Fort Worth,Westridge Area,3,Home Service,Gas Station,Gym / Fitness Center,Event Service,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dog Run
203,Fort Worth,Dorado Ranch,3,Home Service,Playground,Pool,Farmers Market,Field,Flea Market,Fast Food Restaurant,Floating Market,Donut Shop,Farm
204,Fort Worth,Handley,3,Home Service,Art Gallery,Pub,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Donut Shop


In [36]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 4, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Fort Worth,Hulen Heights,4,Construction & Landscaping,Bar,Dry Cleaner,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
51,Fort Worth,South Meadows Crossing,4,Construction & Landscaping,Food Truck,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
91,Fort Worth,Parkview Hills,4,Construction & Landscaping,Park,Video Store,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
160,Fort Worth,Jenkins Heights - Crest Ridge,4,Construction & Landscaping,Cosmetics Shop,Dry Cleaner,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
173,Fort Worth,South Poly,4,Construction & Landscaping,Diner,Dry Cleaner,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm
235,Garland,Charleston Commons,4,Construction & Landscaping,Concert Hall,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
238,Mesquite,Sherwood Forest,4,Construction & Landscaping,Concert Hall,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
319,Irving,Nimitz,4,Construction & Landscaping,Gift Shop,Baseball Field,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
329,Garland,Meadowood,4,Construction & Landscaping,Food,Convenience Store,Cosmetics Shop,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market
346,Irving,Shady Grove,4,Construction & Landscaping,Concert Hall,Flower Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory


In [39]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 5, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Fort Worth,Chapel Creek,5,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
63,Fort Worth,Western Hills North,5,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
105,Fort Worth,Southgate,5,Discount Store,Gym,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Factory,Donut Shop
107,Fort Worth,Woodhaven,5,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
115,Fort Worth,Park Palisades,5,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
117,Fort Worth,Eastern Hills,5,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
210,Fort Worth,Fairhaven,5,Gym,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory


In [40]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 6, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Fort Worth,Alexandra Meadows,6,Park,Pool,Zoo Exhibit,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Farm
24,Fort Worth,The Crossing of Fossil Creek,6,Food & Drink Shop,Convenience Store,Restaurant,Pool,Zoo Exhibit,Event Service,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
36,Fort Worth,Stone Meadow,6,Trail,Pool,Zoo Exhibit,Event Space,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Factory
77,Fort Worth,Beechwood Creek,6,Pool,Zoo Exhibit,Donut Shop,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
82,Fort Worth,Kingsridge,6,Gym,Construction & Landscaping,Roof Deck,Pool,Zoo Exhibit,Dry Cleaner,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
99,Fort Worth,Fox Run,6,Food,Fast Food Restaurant,Pool,IT Services,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
104,Fort Worth,Overton Woods,6,Bar,Pool,Zoo Exhibit,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farm
113,Fort Worth,Lake of River Trails,6,Bakery,Lake,Pool,Factory,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Zoo Exhibit
140,Fort Worth,Mira Vista,6,Golf Course,Playground,Lounge,Pool,Zoo Exhibit,Event Space,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Service
159,Fort Worth,Overton West,6,Bar,Locksmith,Pool,Zoo Exhibit,Factory,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farmers Market


In [41]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 7, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Fort Worth,Marine Park,7,Mexican Restaurant,Convenience Store,Mediterranean Restaurant,Gas Station,Theater,Discount Store,Zoo Exhibit,Event Space,Electronics Store,Ethiopian Restaurant
34,Fort Worth,Worth Heights,7,Mexican Restaurant,Playground,Zoo Exhibit,Donut Shop,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
96,Fort Worth,Burchill,7,Mexican Restaurant,Zoo Exhibit,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
153,Fort Worth,Hubbard Heights,7,Mexican Restaurant,Zoo Exhibit,Food,Floating Market,Flea Market,Field,Fast Food Restaurant,Farmers Market,Farm,Factory
370,Richardson,Waterfall Crossing,7,Mexican Restaurant,Convenience Store,Discount Store,Zoo Exhibit,Factory,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Farmers Market
